# Testing Logistic Regression and Regression Models for Women Involvement
Logistic Regression - Whether a women was involved on a patent level

Regression - Predicting % women involvement on a patent level and GEOID Level
    - Linear Regression
    - Random Forest

__Note- data was slightly adjusted to remove standardization women and men inventors to obtain a more realistc women involvement % below__

Import Libraries:

In [1]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from statistics import mean
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

## Patent Level Work

### Read Data

In [2]:
os.getcwd()

'C:\\Users\\trent\\Documents\\GitHub\\patents22'

In [3]:
os.chdir('C:\\Users\\trent\\Documents\\Capstone')

In [4]:
train_read = pd.read_csv('Fixed_log_standardized_train.csv')

In [5]:
train_read.shape

(134808, 98)

In [6]:
val_read = pd.read_csv('Fixed_log_standardized_test.csv')

In [7]:
train_read_raw = pd.read_csv('Fixed_log_RAW_train.csv')

In [8]:
test_read_raw = pd.read_csv('Fixed_log_RAW_test.csv')

### Organize Patent Train Data: Prep for Model Building

The collapsed code below checks for null values, and removes columns with null values.

In [9]:
for i in range(len(train_read.columns)):
    if train_read.iloc[:,i].isnull().values.any() == True:
        print(train_read.iloc[:,i].name,': ',train_read.iloc[:,i].isna().sum())

In [10]:
for i in range(len(train_read_raw.columns)):
    if train_read_raw.iloc[:,i].isnull().values.any() == True:
        print(train_read_raw.iloc[:,i].name,': ',train_read_raw.iloc[:,i].isna().sum())

In [11]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
train_ = train_read.drop(['patent_num',
                   'assignee',
                   'app_num',
                   'GEOID',
                   # 'assignee_univ_map',
                   # 'Institution',
                   # 'Control',
                   # 'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    # 'ipc_A',
                    # 'ipc_B',
                    # 'ipc_C',
                    # 'ipc_D',
                    # 'ipc_E',
                    # 'ipc_F',
                    # 'ipc_G',
                    # 'ipc_H',
                    # 'num_ipcs',
                    # 'ipc'
                         ],
                   axis = 1)

In [12]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
train_raw = train_read_raw.drop([
                   #  'patent_num',
                   # 'assignee',
                   'app_num',
                   'GEOID',
                   # 'assignee_univ_map',
                   # 'Institution',
                   # 'Control',
                   # 'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    # 'ipc_A',
                    # 'ipc_B',
                    # 'ipc_C',
                    # 'ipc_D',
                    # 'ipc_E',
                    # 'ipc_F',
                    # 'ipc_G',
                    # 'ipc_H',
                    # 'num_ipcs',
                    # 'ipc'
                         ],
                   axis = 1)

In [13]:
train_raw['perc_wi'] = train_raw['women_inventors']/(train_raw['men_inventors']+train_raw['women_inventors'])

In [14]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
val_ = val_read.drop(['patent_num',
                   'assignee',
                   'app_num',
                   'GEOID',
                   # 'assignee_univ_map',
                   # 'Institution',
                   # 'Control',
                   # 'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    # 'ipc_A',
                    # 'ipc_B',
                    # 'ipc_C',
                    # 'ipc_D',
                    # 'ipc_E',
                    # 'ipc_F',
                    # 'ipc_G',
                    # 'ipc_H',
                    # 'num_ipcs',
                    # 'ipc'
                     ],
                   axis = 1)

In [15]:
##drop columns not necessary for model
##dropping ipc and award data as well for now to create model
val_raw = test_read_raw.drop([
                   #  'patent_num',
                   # 'assignee',
                   'app_num',
                   'GEOID',
                   # 'assignee_univ_map',
                   # 'Institution',
                   # 'Control',
                   # 'grant_yr',
                    # 'Award_Amount',
                    # 'Award_by_pop_gt_16_lf',
                    # 'team_size',
                    # 'men_inventors',
                    # 'ipc_A',
                    # 'ipc_B',
                    # 'ipc_C',
                    # 'ipc_D',
                    # 'ipc_E',
                    # 'ipc_F',
                    # 'ipc_G',
                    # 'ipc_H',
                    # 'num_ipcs',
                    # 'ipc'
                     ],
                   axis = 1)

In [16]:
val_raw['perc_wi'] = val_raw['women_inventors']/(val_raw['men_inventors']+val_raw['women_inventors'])

In [17]:
train_raw['perc_wi'].value_counts()

0.000000    906284
0.500000     64820
0.333333     58642
0.250000     42364
1.000000     33197
             ...  
0.233333         1
0.065574         1
0.137931         1
0.520000         1
0.404762         1
Name: perc_wi, Length: 154, dtype: int64

In [18]:
train_raw[train_raw['perc_wi'].isnull()]

,app_yr,patent_num,assignee,team_size,men_inventors,women_inventors,women_involved,Over25_Less9Grade,Over25_SomeHS,Over25_HSGrad,...,ap_by_est,est_by_pop_gt_16_lf,apps_by_pop_gt_16_lf,Wages_by_pop_gt_16_lf,Award_by_pop_gt_16_lf,r1_involved,r2_involved,special_focus_involved,any_R_uni_involved,perc_wi
24,2014,0,1,1,0,0,0,4.5,8.4,26.3,...,518.466322,0.051951,0.021566,33.985626,9.334419,0,0,0,0,NaN
221,2012,1,2,1,0,0,0,3.9,6.7,30.7,...,476.844625,0.037794,0.009442,41.991776,47.465511,0,0,0,0,NaN
223,2010,1,1,1,0,0,0,6.7,6.4,15.9,...,1644.687829,0.048169,0.014315,87.241578,25.681263,0,0,0,0,NaN
322,2010,1,1,1,0,0,0,5.7,6.2,19.6,...,763.785946,0.041574,0.012614,37.413798,22.546429,0,0,0,0,NaN
346,2010,1,1,1,0,0,0,2.8,6.9,26.1,...,565.455645,0.044734,0.011908,33.103770,20.133450,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212651,2013,1,1,1,0,0,0,8.4,7.5,17.4,...,812.117927,0.055706,0.017653,54.148898,19.085219,0,0,0,0,NaN
1212688,2014,1,1,1,0,0,0,6.9,6.3,13.3,...,2185.595219,0.111933,0.046456,287.479167,322.235625,0,0,0,0,NaN
1212698,2014,1,1,1,0,0,0,6.4,6.1,15.1,...,2154.077309,0.048212,0.015417,111.984429,17.632490,0,0,0,0,NaN
1212875,2013,1,2,1,0,0,0,7.2,6.0,21.8,...,733.759343,0.063498,0.020344,57.218067,21.750768,0,0,0,0,NaN


In [19]:
for i in range(len(train_raw.columns)):
    if train_raw.iloc[:,i].isnull().values.any() == True:
        print(train_raw.iloc[:,i].name,': ',train_raw.iloc[:,i].isna().sum())

perc_wi :  16456


__Some patents include 0 men and women inventors, causing some perc women % to be null, these are later dropped__

Numeric Feature List Below:

In [20]:
numeric = [
 # 'num_ipcs',
 'team_size',
 # 'men_inventors',
 # 'women_inventors',
 'Over25_Less9Grade',
 'Over25_SomeHS',
 'Over25_HSGrad',
 'Over25_SomeCollege',
 'Over25_Assosc',
 'Over25_Bach',
 'Over25_Grad',
 'ap_by_est',
 'pop_gt_16_lf',
 'est_by_pop_gt_16_lf',
 'Total_employment',
 'Pop_Est',
 'Population_Estimate',
 'Population_persons',
 'Earnings_by_place_of_work',
 'Employee_and_self-employed_contributions_for_government_social_insurance',
 'Employer_contributions_for_employee_pension_and_insurance_funds',
 'Employer_contributions_for_government_social_insurance',
 'Equals_Net_earnings_by_place_of_residence',
 'Farm_income',
 'Farm_proprietors_income',
 'Less_Contributions_for_government_social_insurance',
 'Nonfarm_personal_income',
 'Nonfarm_proprietors_income',
 'Per_capita_personal_income_dollars',
 'Personal_income_thousands_of_dollars',
 'Plus_Adjustment_for_residence',
 'Plus_Dividends_interest_and_rent',
 'Plus_Personal_current_transfer_receipts',	
 'Proprietors_employment',
 'Proprietors_income',
 'Supplements_to_wages_and_salaries',
 'Wage_and_salary_employment',
 'Wages_and_salaries',
 'Applications',
 # 'Award_Amount',
 # 'GDP_by_labor_force',
 'apps_by_pop_gt_16_lf',
 'Wages_by_pop_gt_16_lf',
 # 'Award_by_pop_gt_16_lf'
]

Categorical Feature List Below:

In [21]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
'women_involved'
]

#### One Hot Encode Categoricals
All already binary except region

In [22]:
train = pd.concat([train_.drop('bea_region', axis =1), pd.get_dummies(train_.bea_region)], axis = 1)
val = pd.concat([val_.drop('bea_region', axis =1), pd.get_dummies(val_.bea_region)], axis = 1)

raw_train = pd.concat([train_raw.drop('bea_region', axis =1), pd.get_dummies(train_raw.bea_region)], axis = 1)
raw_val = pd.concat([val_raw.drop('bea_region', axis =1), pd.get_dummies(val_raw.bea_region)], axis = 1)

#### Creating X and Y train, val sets for both Logistic Regression and Linear Regression on a Patent Level

In [23]:
##binary class for women involvement
###train
y_wi= train['women_involved']
x_wi= train.drop(['women_involved','women_inventors', 'men_inventors'], axis = 1)

##val
x_val_wi = val.drop(['women_involved','women_inventors', 'men_inventors'], axis = 1)
y_val_wi= val['women_involved']

##women percentage
###train
y_wi_perc = raw_train['perc_wi'].dropna()
x_wi_perc= raw_train.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

##val
y_val_wi_perc = raw_val['perc_wi'].dropna()
x_val_wi_perc= raw_val.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

In [24]:
x_wi.shape

(134808, 98)

In [25]:
x_wi_perc.shape

(1196811, 100)

### Logistic Regression Models

In [26]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
# 'already_granted',
# 'Control',
'r1_involved',
'r2_involved',
'special_focus_involved',
# 'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
# 'women_involved'
'Far West',
'Great Lakes', 
'Mideast', 
'New England', 
'Plains', 
'Rocky Mountain',
'Southeast', 
'Southwest'
]

In [27]:
# x_wi.columns

In [28]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi[['r1_involved','Over25_Grad']], y_wi)
# evaluate the model
yhat = model.predict(x_val_wi[['r1_involved','Over25_Grad']])
y_hat2 = (model.predict_proba(x_val_wi[['r1_involved','Over25_Grad']])[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))

[[   203 922537]
 [    43 290484]]
              precision    recall  f1-score   support

           0       0.83      0.00      0.00    922740
           1       0.24      1.00      0.39    290527

    accuracy                           0.24   1213267
   macro avg       0.53      0.50      0.19   1213267
weighted avg       0.68      0.24      0.09   1213267



In [29]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi[categorical], y_wi)
# evaluate the model
yhat = model.predict(x_val_wi[categorical])
y_hat2 = (model.predict_proba(x_val_wi[categorical])[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))

[[856206  66534]
 [254469  36058]]
              precision    recall  f1-score   support

           0       0.77      0.93      0.84    922740
           1       0.35      0.12      0.18    290527

    accuracy                           0.74   1213267
   macro avg       0.56      0.53      0.51   1213267
weighted avg       0.67      0.74      0.68   1213267



In [30]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.5).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[917698   5042]
 [288690   1837]]
              precision    recall  f1-score   support

           0       0.76      0.99      0.86    922740
           1       0.27      0.01      0.01    290527

    accuracy                           0.76   1213267
   macro avg       0.51      0.50      0.44   1213267
weighted avg       0.64      0.76      0.66   1213267

col_0            False  True
women_involved              
0               917698  5042
1               288690  1837


In [31]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.4).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[917698   5042]
 [288690   1837]]
              precision    recall  f1-score   support

           0       0.76      0.99      0.86    922740
           1       0.27      0.01      0.01    290527

    accuracy                           0.76   1213267
   macro avg       0.51      0.50      0.44   1213267
weighted avg       0.64      0.76      0.66   1213267

col_0            False  True
women_involved              
0               917698  5042
1               288690  1837


In [32]:
model = LogisticRegression(solver='liblinear')
logreg = model.fit(x_wi, y_wi)
# evaluate the model
yhat = model.predict(x_val_wi)
y_hat2 = (model.predict_proba(x_val_wi)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
# evaluate predictions
# accuracy = model.score(x_wi[categorical], y_val_wi)
# print('Accuracy: %.2f' % (accuracy*100))
# print(metrics.confusion_matrix(y_val_wi, yhat))
# print(classification_report(y_val_wi, yhat))
print(metrics.confusion_matrix(y_val_wi, y_hat2))
print(classification_report(y_val_wi, y_hat2))
print(pd.crosstab(y_val_wi, y_hat2))

[[917698   5042]
 [288690   1837]]
              precision    recall  f1-score   support

           0       0.76      0.99      0.86    922740
           1       0.27      0.01      0.01    290527

    accuracy                           0.76   1213267
   macro avg       0.51      0.50      0.44   1213267
weighted avg       0.64      0.76      0.66   1213267

col_0            False  True
women_involved              
0               917698  5042
1               288690  1837


In [33]:
# app_yr', 'team_size', 'Over25_Less9Grade', 'Over25_SomeHS',
#        'Over25_HSGrad', 'Over25_SomeCollege', 'Over25_Assosc', 'Over25_Bach',
#        'Over25_Grad', 'Agriculture_Forestry_Fishing_Hunting',
#        'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction',
#        'Manufacturing', 'Wholesale_Trade', 'Retail_Trade',
#        'Transportation_Warehousing', 'Information', 'Finance_Insurance',
#        'Real_Estate_Rental_Leasing',
#        'Professional_Scientific_and_Technical_Services',
#        'Management_of_Companies_Enterprises',
#        'Administrative_Support_Waste_Management_Remediation_Services',
#        'Educational_Services', 'Health_Care_Social_Assistance',
#        'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services',
#        'Other_Services_except_Public_Administration', 'qp1', 'ap', 'est',
#        'Agriculture_Forestry_Fishing_Hunting_base',
#        'Mining_Quarrying_and_Oil_Gas_Extraction_base', 'Utilities_base',
#        'Construction_base', 'Manufacturing_base', 'Wholesale_Trade_base',
#        'Retail_Trade_base', 'Transportation_Warehousing_base',
#        'Information_base', 'Finance_Insurance_base',
#        'Real_Estate_Rental_Leasing_base',
#        'Professional_Scientific_and_Technical_Services_base',
#        'Management_of_Companies_Enterprises_base',
#        'Administrative_Support_Waste_Management_Remediation_Services_base',
#        'Educational_Services_base', 'Health_Care_Social_Assistance_base',
#        'Arts_Entertainment_and_Recreation_base',
#        'Accommodation_Food_Services_base',
#        'Other_Services_except_Public_Administration_base', 'GDP', 'pop_gt_16',
#        'pop_gt_16_lf', 'pop_gt_16_lf_c', 'Pop_Est',
#        'Earnings_by_place_of_work',
#        'Employee_and_self-employed_contributions_for_government_social_insurance',
#        'Employer_contributions_for_employee_pension_and_insurance_funds',
#        'Employer_contributions_for_government_social_insurance',
#        'Equals_Net_earnings_by_place_of_residence', 'Farm_income',
#        'Farm_proprietors_income',
#        'Less_Contributions_for_government_social_insurance',
#        'Nonfarm_personal_income', 'Nonfarm_proprietors_income',
#        'Per_capita_personal_income_dollars',
#        'Personal_income_thousands_of_dollars', 'Plus_Adjustment_for_residence',
#        'Plus_Dividends_interest_and_rent',
#        'Plus_Personal_current_transfer_receipts', 'Population_persons',
#        'Proprietors_employment', 'Proprietors_income',
#        'Supplements_to_wages_and_salaries', 'Total_employment',
#        'Wage_and_salary_employment', 'Wages_and_salaries',
#        'Population_Estimate', 'Applications', 'Award_Amount',
#        'GDP_by_labor_force', 'ap_by_est', 'est_by_pop_gt_16_lf',
#        'apps_by_pop_gt_16_lf', 'Wages_by_pop_gt_16_lf',
#        'Award_by_pop_gt_16_lf', 'r1_involved', 'r2_involved',
#        'special_focus_involved', 'any_R_uni_involved', 'Far West',
#        'Great Lakes', 'Mideast', 'New England', 'Plains', 'Rocky Mountain',
#        'Southeast', 'Southwest'

#### Exploring Different Model Combinations, Including Polynomial Features

In [34]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_roc_curve
from sklearn_pandas import DataFrameMapper

In [35]:
# def log_reg(features, x_train, y_train, x_test, y_test):
#     x_test = x_test[features]
#     model = LogisticRegression(solver='liblinear')
#     logreg = model.fit(x_train[features], y_train)
#     yhat = model.predict_proba(x_test)[:,1]
#     auc = metrics.roc_auc_score(y_test, yhat).round(3)
#     lst_dict.append({'auc':auc,'features':features})
#     return lst_dict

In [36]:
def log_reg(features, x_train, y_train, x_test, y_test, polynomials = None, degree = 1):

    if polynomials == None:
        x_test = x_test[features]
        model = LogisticRegression(solver='liblinear')
        logreg = model.fit(x_train[features], y_train)
        yhat = model.predict_proba(x_test)[:,1]
        auc = metrics.roc_auc_score(y_test, yhat).round(3)
        lst_dict.append({'auc':auc,'features':features, 'degree': degree, 
                         'polynomial_features':polynomials})
        return lst_dict
    
    else:
        x_test = x_test[features]
        x_train = x_train[features]
        poly = DataFrameMapper([(polynomials, 
                               PolynomialFeatures(degree))
                               ])
        x_train = poly.fit_transform(x_train)
        x_test = poly.fit_transform(x_test)
        model = LogisticRegression(solver='liblinear')
        logreg = model.fit(x_train, y_train)
        yhat = model.predict_proba(x_test)[:,1]
        auc = metrics.roc_auc_score(y_test, yhat).round(3)
        lst_dict.append({'auc':auc,'features':features, 'degree': degree, 
                         'polynomial_features':polynomials})
        return lst_dict

##### Feature Subsets

In [37]:
regions = ['Far West','Great Lakes', 'Mideast', 'New England', 'Plains', 'Rocky Mountain', 'Southeast', 'Southwest']

In [38]:
university = ['r1_involved', 'r2_involved',
       'special_focus_involved']

In [39]:
economics = ['Earnings_by_place_of_work',
       'Employee_and_self-employed_contributions_for_government_social_insurance',
       'Employer_contributions_for_employee_pension_and_insurance_funds',
       'Employer_contributions_for_government_social_insurance',
       'Equals_Net_earnings_by_place_of_residence', 'Farm_income',
       'Farm_proprietors_income',
       'Less_Contributions_for_government_social_insurance',
       'Nonfarm_personal_income', 'Nonfarm_proprietors_income',
       'Per_capita_personal_income_dollars',
       'Personal_income_thousands_of_dollars', 'Plus_Adjustment_for_residence',
       'Plus_Dividends_interest_and_rent',
       'Plus_Personal_current_transfer_receipts', 'Population_persons',
       'Proprietors_employment', 'Proprietors_income',
       'Supplements_to_wages_and_salaries', 'Total_employment',
       'Wage_and_salary_employment', 'Wages_and_salaries',
       'Population_Estimate', 'Applications', 'Award_Amount',
       'GDP_by_labor_force', 'ap_by_est', 'est_by_pop_gt_16_lf',
       'apps_by_pop_gt_16_lf', 'Wages_by_pop_gt_16_lf',
       'Award_by_pop_gt_16_lf', 'GDP']

In [40]:
education = ['Over25_HSGrad', 'Over25_SomeCollege', 'Over25_Assosc', 'Over25_Bach',
       'Over25_Grad']

In [41]:
naics = ['Agriculture_Forestry_Fishing_Hunting',
       'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction',
       'Manufacturing', 'Wholesale_Trade', 'Retail_Trade',
       'Transportation_Warehousing', 'Information', 'Finance_Insurance',
       'Real_Estate_Rental_Leasing',
       'Professional_Scientific_and_Technical_Services',
       'Management_of_Companies_Enterprises',
       'Administrative_Support_Waste_Management_Remediation_Services',
       'Educational_Services', 'Health_Care_Social_Assistance',
       'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services',
       'Other_Services_except_Public_Administration']

In [42]:
top_4_chi = ['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved']

In [43]:
top_5_chi = ['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Southwest']

In [44]:
top_4_chi

['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved']

In [45]:
dicts = [{'features':top_4_chi + ['Over25_Bach'], 'polynomials':['Over25_Bach']},
         {'features':top_5_chi + ['Over25_Bach'], 'polynomials':['Over25_Bach']},
         {'features':top_4_chi + education, 'polynomials':education},
         {'features':top_4_chi + ['Over25_Bach', 'Award_by_pop_gt_16_lf'], 
          'polynomials':['Over25_Bach', 'Award_by_pop_gt_16_lf']},
         {'features':top_5_chi + ['Over25_Bach', 'Award_by_pop_gt_16_lf'], 
          'polynomials':['Over25_Bach', 'Award_by_pop_gt_16_lf']},
         {'features':top_4_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'], 
          'polynomials':['Over25_Grad', 'Award_by_pop_gt_16_lf']},
         {'features':top_5_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'], 
          'polynomials':['Over25_Grad', 'Award_by_pop_gt_16_lf']},
         {'features':top_4_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics, 
          'polynomials':['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics},
         {'features': top_5_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics,
          'polynomials':['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
         # {'features':, 'polynomials':},
        ]

In [46]:
features_polynomials = pd.DataFrame()
features_polynomials = features_polynomials.append(dicts)

C:\Users\trent\AppData\Local\Temp\ipykernel_21400\2157273320.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_polynomials = features_polynomials.append(dicts)


In [47]:
features_polynomials

,features,polynomials
0,"[r1_involved, special_focus_involved, Rocky Mo...",[Over25_Bach]
1,"[r1_involved, special_focus_involved, Rocky Mo...",[Over25_Bach]
2,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_HSGrad, Over25_SomeCollege, Over25_Ass..."
3,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Bach, Award_by_pop_gt_16_lf]"
4,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Bach, Award_by_pop_gt_16_lf]"
5,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Grad, Award_by_pop_gt_16_lf]"
6,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Grad, Award_by_pop_gt_16_lf]"
7,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Grad, Award_by_pop_gt_16_lf, Agricultu..."
8,"[r1_involved, special_focus_involved, Rocky Mo...","[Over25_Grad, Award_by_pop_gt_16_lf, Agricultu..."


In [48]:
features_polynomials['features'].loc[0]

['r1_involved',
 'special_focus_involved',
 'Rocky Mountain',
 'r2_involved',
 'Over25_Bach']

In [49]:
range(len(features_polynomials))

range(0, 9)

In [74]:
lst_dict = []
for i in range(len(features_polynomials)):
    log_reg(features_polynomials['features'].loc[i],
            x_wi, y_wi, x_val_wi, y_val_wi,
            polynomials = None, 
            degree = 1)
    
# df = pd.DataFrame(lst_dict)
df1 = pd.DataFrame(lst_dict)
df = pd.concat([df,df1]).reset_index().drop('index', axis =1)

In [77]:
df.style.set_properties(subset=['features'], **{'width': '500px'})

,auc,features,degree,polynomial_features
0,0.472000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Over25_Bach']",2,['Over25_Bach']
1,0.472000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Southwest', 'Over25_Bach']",2,['Over25_Bach']
2,0.498000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Over25_HSGrad', 'Over25_SomeCollege', 'Over25_Assosc', 'Over25_Bach', 'Over25_Grad']",2,"['Over25_HSGrad', 'Over25_SomeCollege', 'Over25_Assosc', 'Over25_Bach', 'Over25_Grad']"
3,0.469000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Over25_Bach', 'Award_by_pop_gt_16_lf']",2,"['Over25_Bach', 'Award_by_pop_gt_16_lf']"
4,0.469000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Southwest', 'Over25_Bach', 'Award_by_pop_gt_16_lf']",2,"['Over25_Bach', 'Award_by_pop_gt_16_lf']"
5,0.458000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Over25_Grad', 'Award_by_pop_gt_16_lf']",2,"['Over25_Grad', 'Award_by_pop_gt_16_lf']"
6,0.458000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Southwest', 'Over25_Grad', 'Award_by_pop_gt_16_lf']",2,"['Over25_Grad', 'Award_by_pop_gt_16_lf']"
7,0.512000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Over25_Grad', 'Award_by_pop_gt_16_lf', 'Agriculture_Forestry_Fishing_Hunting', 'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction', 'Manufacturing', 'Wholesale_Trade', 'Retail_Trade', 'Transportation_Warehousing', 'Information', 'Finance_Insurance', 'Real_Estate_Rental_Leasing', 'Professional_Scientific_and_Technical_Services', 'Management_of_Companies_Enterprises', 'Administrative_Support_Waste_Management_Remediation_Services', 'Educational_Services', 'Health_Care_Social_Assistance', 'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services', 'Other_Services_except_Public_Administration']",2,"['Over25_Grad', 'Award_by_pop_gt_16_lf', 'Agriculture_Forestry_Fishing_Hunting', 'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction', 'Manufacturing', 'Wholesale_Trade', 'Retail_Trade', 'Transportation_Warehousing', 'Information', 'Finance_Insurance', 'Real_Estate_Rental_Leasing', 'Professional_Scientific_and_Technical_Services', 'Management_of_Companies_Enterprises', 'Administrative_Support_Waste_Management_Remediation_Services', 'Educational_Services', 'Health_Care_Social_Assistance', 'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services', 'Other_Services_except_Public_Administration']"
8,0.512000,"['r1_involved', 'special_focus_involved', 'Rocky Mountain', 'r2_involved', 'Southwest', 'Over25_Grad', 'Award_by_pop_gt_16_lf', 'Agriculture_Forestry_Fishing_Hunting', 'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction', 'Manufacturing', 'Wholesale_Trade', 'Retail_Trade', 'Transportation_Warehousing', 'Information', 'Finance_Insurance', 'Real_Estate_Rental_Leasing', 'Professional_Scientific_and_Technical_Services', 'Management_of_Companies_Enterprises', 'Administrative_Support_Waste_Management_Remediation_Services', 'Educational_Services', 'Health_Care_Social_Assistance', 'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services', 'Other_Services_except_Public_Administration']",2,"['Over25_Grad', 'Award_by_pop_gt_16_lf', 'Agriculture_Forestry_Fishing_Hunting', 'Mining_Quarrying_and_Oil_Gas_Extraction', 'Utilities', 'Construction', 'Manufacturing', 'Wholesale_Trade', 'Retail_Trade', 'Transportation_Warehousing', 'Information', 'Finance_Insurance', 'Real_Estate_Rental_Leasing', 'Professional_Scientific_and_Technical_Services', 'Management_of_Companies_Enterprises', 'Administrative_Support_Waste_Management_Remediation_Services', 'Educational_Services', 'Health_Care_Social_Assistance', 'Arts_Entertainment_and_Recreation', 'Accommodation_Food_Services', 'Other_Services_except_Public_Administration']"
9,0.547000,"['r1_invol

In [76]:
pd.set_option('display.width', 1000)

In [ ]:
pipe = Pipeline([('polynomial_features',poly), ('logistic_regression',lr)])
pipe.fit(x_wi[top_4_chi+ ['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_wi)
pipe.score(x_val_wi[top_4_chi+ ['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)
plot_roc_curve(pipe, x_val_wi[top_4_chi+ ['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)

In [ ]:
pipe = Pipeline([('polynomial_features',poly), ('logistic_regression',lr)])
pipe.fit(x_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_wi)
pipe.score(x_val_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)
plot_roc_curve(pipe, x_val_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)

In [ ]:
poly = PolynomialFeatures(degree = 3, interaction_only=False, include_bias=False)
pipe = Pipeline([('polynomial_features',poly), ('logistic_regression',lr)])
pipe.fit(x_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_wi)
pipe.score(x_val_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)
plot_roc_curve(pipe, x_val_wi[['Over25_Bach', 'Award_by_pop_gt_16_lf']], y_val_wi)

### Linear Regression - Individual Patent

##### Exploring Different Model Combinations

In [78]:
def lin_reg(features, x_train, y_train, x_test, y_test):
    x_test = x_test[features]
    model = LogisticRegression(solver='liblinear')
    reg = LinearRegression().fit(x_train[features], y_train)
    r_score = reg.score(x_test[features], y_test)
    lst_dict.append({'r_squared':r_score,'features':features})
    return lst_dict

In [ ]:
from sklearn.linear_model import LinearRegression
predictors = [
    'r1_involved',
    'Over25_HSGrad',
    'r2_involved',
    'Agriculture_Forestry_Fishing_Hunting',
    'Mining_Quarrying_and_Oil_Gas_Extraction',
    'Utilities_base',
    'Construction_base',
    'Manufacturing',
    'Wholesale_Trade',
    'Retail_Trade',
    'Transportation_Warehousing',
    'Information',
    'Finance_Insurance',
    'Real_Estate_Rental_Leasing',
    'Professional_Scientific_and_Technical_Services',
    'Management_of_Companies_Enterprises',
    'Administrative_Support_Waste_Management_Remediation_Services',
    'Educational_Services',
    'Health_Care_Social_Assistance',
    'Arts_Entertainment_and_Recreation',
    'Accommodation_Food_Services',
    'Other_Services_except_Public_Administration',
    ]
reg = LinearRegression().fit(x_wi_perc[predictors], y_wi_perc)
reg.score(x_val_wi_perc[predictors], y_val_wi_perc)

In [79]:
feature_list = [
               top_4_chi + ['Over25_Bach'],
               top_5_chi + ['Over25_Bach'],
               top_4_chi + education,
               top_4_chi + ['Over25_Bach', 'Award_by_pop_gt_16_lf'],
               top_5_chi + ['Over25_Bach', 'Award_by_pop_gt_16_lf'],
               top_4_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'],
               top_5_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'], 
               top_4_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics,
               top_5_chi + ['Over25_Grad', 'Award_by_pop_gt_16_lf'] + naics,
               economics + regions + university
              ]

In [80]:
# df_lr = pd.DataFrame(columns = [['features','r_squared']])
lst_dict = []
for i in feature_list:
    lin_reg(i, x_wi_perc, y_wi_perc, x_val_wi_perc, y_val_wi_perc)


In [81]:
df_lr = pd.DataFrame(lst_dict)

In [82]:
df_lr

,r_squared,features
0,0.006926,"[r1_involved, special_focus_involved, Rocky Mo..."
1,0.007155,"[r1_involved, special_focus_involved, Rocky Mo..."
2,0.008067,"[r1_involved, special_focus_involved, Rocky Mo..."
3,0.007225,"[r1_involved, special_focus_involved, Rocky Mo..."
4,0.007421,"[r1_involved, special_focus_involved, Rocky Mo..."
5,0.007936,"[r1_involved, special_focus_involved, Rocky Mo..."
6,0.007989,"[r1_involved, special_focus_involved, Rocky Mo..."
7,0.010264,"[r1_involved, special_focus_involved, Rocky Mo..."
8,0.010279,"[r1_involved, special_focus_involved, Rocky Mo..."
9,0.010398,"[Earnings_by_place_of_work, Employee_and_self-..."


### Random Forest - Individual Patent

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# rf.fit(x_wi_perc[predictors], y_wi_perc)

In [ ]:
# predictions = rf.predict(x_val_wi_perc[predictors])
# errors = abs(predictions - y_val_wi_perc)

In [ ]:
# print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# from sklearn.metrics import mean_squared_error

# mean_squared_error(y_val_wi_perc, predictions)

## GEOID Level Work - Regression for % Women Involvement

### Train Data on GEOID Level: Prep for Model Building

In [83]:
train_read_lr = pd.read_csv('Fix_lr_RAW_train.csv')

In [84]:
train_read_lr.shape

(12377, 105)

In [85]:
val_read_lr = pd.read_csv('Fixed_lr_RAW_test.csv')

The collapsed code below checks for null values, and removes columns with null values.

In [86]:
for i in range(len(train_read_lr.columns)):
    if train_read_lr.iloc[:,i].isnull().values.any() == True:
        print(train_read_lr.iloc[:,i].name,': ',train_read_lr.iloc[:,i].isna().sum())

In [87]:
train_read_lr['perc_wi'] = train_read_lr['women_inventors']/(train_read_lr['men_inventors']+train_read_lr['women_inventors'])

In [88]:
val_read_lr['perc_wi'] = val_read_lr['women_inventors']/(val_read_lr['men_inventors']+val_read_lr['women_inventors'])

Numeric Feature List Below:

In [89]:
numeric = [
 # 'num_ipcs',
 'team_size',
 # 'men_inventors',
 # 'women_inventors',
 'Over25_Less9Grade',
 'Over25_SomeHS',
 'Over25_HSGrad',
 'Over25_SomeCollege',
 'Over25_Assosc',
 'Over25_Bach',
 'Over25_Grad',
 'ap_by_est',
 'pop_gt_16_lf',
 'est_by_pop_gt_16_lf',
 'Total_employment',
 'Pop_Est',
 'Population_Estimate',
 'Population_persons',
 'Earnings_by_place_of_work',
 'Employee_and_self-employed_contributions_for_government_social_insurance',
 'Employer_contributions_for_employee_pension_and_insurance_funds',
 'Employer_contributions_for_government_social_insurance',
 'Equals_Net_earnings_by_place_of_residence',
 'Farm_income',
 'Farm_proprietors_income',
 'Less_Contributions_for_government_social_insurance',
 'Nonfarm_personal_income',
 'Nonfarm_proprietors_income',
 'Per_capita_personal_income_dollars',
 'Personal_income_thousands_of_dollars',
 'Plus_Adjustment_for_residence',
 'Plus_Dividends_interest_and_rent',
 'Plus_Personal_current_transfer_receipts',	
 'Proprietors_employment',
 'Proprietors_income',
 'Supplements_to_wages_and_salaries',
 'Wage_and_salary_employment',
 'Wages_and_salaries',
 'Applications',
 # 'Award_Amount',
 # 'GDP_by_labor_force',
 'apps_by_pop_gt_16_lf',
 'Wages_by_pop_gt_16_lf',
 # 'Award_by_pop_gt_16_lf'
]

Categorical Feature List Below:

In [90]:
categorical = [
# 'ipc',
# 'ipc_A',
# 'ipc_B',
# 'ipc_C',
# 'ipc_D',
# 'ipc_E',
# 'ipc_F',
# 'ipc_G',
# 'ipc_H',
'already_granted',
# 'Control',
'r1',
'r2',
'special_focus',
'bea_region',
'Agriculture_Forestry_Fishing_Hunting_base',
'Mining_Quarrying_and_Oil_Gas_Extraction_base',
'Utilities_base',
'Construction_base',
'Manufacturing_base',
'Wholesale_Trade_base',
'Retail_Trade_base',
'Transportation_Warehousing_base',
'Information_base',
'Finance_Insurance_base',
'Real_Estate_Rental_Leasing_base',
'Professional_Scientific_and_Technical_Services_base',
'Management_of_Companies_Enterprises_base',
'Administrative_Support_Waste_Management_Remediation_Services_base',
'Educational_Services_base',
'Health_Care_Social_Assistance_base',
'Arts_Entertainment_and_Recreation_base',
'Accommodation_Food_Services_base',
'Other_Services_except_Public_Administration_base',
'women_involved'
]

#### One Hot Encode Categoricals
All already binary except region

In [91]:
train_lr = pd.concat([train_read_lr.drop('bea_region', axis =1), pd.get_dummies(train_read_lr.bea_region)], axis = 1)
val_lr = pd.concat([val_read_lr.drop('bea_region', axis =1), pd.get_dummies(val_read_lr.bea_region)], axis = 1)

In [92]:

##women percentage
###train
y_wi_perc_agg = train_lr['perc_wi'].dropna()
x_wi_perc_agg = train_lr.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

##val
y_val_wi_perc_agg = val_lr['perc_wi'].dropna()
x_val_wi_perc_agg = val_lr.dropna().drop(['women_involved','women_inventors', 'men_inventors','perc_wi'], axis = 1)
#x - same as above

In [93]:
train_lr

,app_yr,GEOID,app_num,team_size,men_inventors,women_inventors,women_involved,r1_involved,r2_involved,special_focus_involved,...,boxcox_apps,perc_wi,Far West,Great Lakes,Mideast,New England,Plains,Rocky Mountain,Southeast,Southwest
0,2016,36061,2576,7996,6276,1208,835,114,2,38,...,3.259197,0.161411,0,0,1,0,0,0,0,0
1,2013,51089,7,39,33,6,6,0,0,0,...,1.513593,0.153846,0,0,0,0,0,0,1,0
2,2018,10005,8,11,10,1,1,0,0,0,...,1.591143,0.090909,0,0,1,0,0,0,0,0
3,2010,24013,10,20,18,2,2,0,0,0,...,1.714655,0.100000,0,0,1,0,0,0,0,0
4,2015,53035,13,23,23,0,0,0,0,0,...,1.850671,0.000000,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12372,2010,29225,3,4,2,0,0,0,0,0,...,0.950646,0.000000,0,0,0,0,1,0,0,0
12373,2013,51041,22,37,33,3,3,0,0,0,...,2.096023,0.083333,0,0,0,0,0,0,1,0
12374,2016,40017,1,1,1,0,0,0,0,0,...,0.000000,0.000000,0,0,0,0,0,0,0,1
12375,2011,13129,7,22,20,2,1,0,0,0,...,1.513593,0.090909,0,0,0,0,0,0,1,0


### Linear Regression Model

In [94]:
# df_lr = pd.DataFrame(columns = [['features','r_squared']])
lst_dict = []
for i in feature_list:
    lin_reg(i, x_wi_perc_agg, y_wi_perc_agg, x_val_wi_perc_agg, y_val_wi_perc_agg)

In [95]:
df_lr_agg = pd.DataFrame(lst_dict)

In [96]:
df_lr_agg

,r_squared,features
0,0.004237,"[r1_involved, special_focus_involved, Rocky Mo..."
1,0.004722,"[r1_involved, special_focus_involved, Rocky Mo..."
2,0.012438,"[r1_involved, special_focus_involved, Rocky Mo..."
3,0.004239,"[r1_involved, special_focus_involved, Rocky Mo..."
4,0.004717,"[r1_involved, special_focus_involved, Rocky Mo..."
5,0.013176,"[r1_involved, special_focus_involved, Rocky Mo..."
6,0.014004,"[r1_involved, special_focus_involved, Rocky Mo..."
7,0.007942,"[r1_involved, special_focus_involved, Rocky Mo..."
8,0.008633,"[r1_involved, special_focus_involved, Rocky Mo..."
9,0.007176,"[Earnings_by_place_of_work, Employee_and_self-..."


#### Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# rf.fit(x_wi_perc_lr[predictors], y_wi_perc_lr)

In [ ]:
# predictions = rf.predict(x_val_wi_perc_lr[predictors])
# errors = abs(predictions - y_val_wi_perc_lr)

In [ ]:
# print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')